In [7]:
import os
from lxml.html import parse
from urllib import urlopen, urlretrieve
from urlparse import urljoin
from subprocess import call
from os.path import exists
import pandas as pd

In [8]:
PDF_TO_TEXT = 'pdftotext'

In [9]:
base = 'http://eci.nic.in/eci_main1/ElectionStatistics.aspx'
tree = parse(urlopen(base))

In [10]:
files = set()
def download(year, link):
    """Download a year's election results and convert it into text"""
    pdf_file = os.path.join('raw', year + '.pdf')
    if not exists(pdf_file):
        urlretrieve(urljoin(base, link), pdf_file)
    text_file = pdf_file.replace('.pdf', '.txt')
    if not exists(text_file):
        call([PDF_TO_TEXT, '-layout', pdf_file, text_file])
        
download('2009', 'http://eci.nic.in/eci_main/archiveofge2009/Stats/VOLI/25_ConstituencyWiseDetailedResult.pdf')
download('2014', 'http://eci.nic.in/eci_main/archiveofge2014/33%20-%20Constituency%20wise%20detailed%20result.pdf')

IOError: [Errno 2] No such file or directory: 'raw/2009.pdf'

In [11]:
import re

In [12]:
fieldlist = {
    '2009.txt': ['#', 'NAME', 'SEX', 'AGE', 'CATEGORY', 'PARTY', 'GENERAL VOTES', 'POSTAL VOTES', 'VOTES', '% ELECTORS', '% VOTES'],
    '2014.txt': ['#', 'NAME', 'SEX', 'AGE', 'CATEGORY', 'PARTY', 'SYMBOL', 'GENERAL VOTES', 'POSTAL VOTES', 'VOTES', '% ELECTORS', '% VOTES']
}

In [13]:
def old_text_parse(filename):
    #regex
    re_state = re.compile(r'^[A-Z][A-Za-z& ]+$')
    re_electors = re.compile(r'Total Electors *(\d+)(.*)')
    re_constituency = re.compile(r'Constituency *:? *(\d+) *\.? *(.*)', re.IGNORECASE)
    re_name = re.compile(r'^\d+ *\. *')
    re_scst = re.compile(r' *\((SC|ST)\)')
    
    #fields
    fields = fieldlist[filename]
    
    results, electors = [], {}
    state, constituency = None, None
    for line in open('raw/'+filename):
        match = re_constituency.match(line)
        if match:
            constituency = match.group(2).split('  ')[0].upper().strip()
            electors[state, constituency] = match.group().split()[-2]
            continue
            
        match = re_state.match(line)
        if match and len(line) < 50:
            state = line.strip().upper()
            continue
            
        parts = re.split('  +', line.strip())
        if len(parts) == len(fields):
            row = dict(zip(fields, parts))
            row['#'] = row['NAME'].split()[0]
            row['NAME'] = ' '.join(row['NAME'].split()[1:])
        else:
            continue
        
        row['STATE'] = state
        row['PC'] = constituency
        results.append(row)
        
    results = pd.DataFrame(results).set_index(['STATE', 'PC'])
    results['YEAR'] = filename.split('.')[0]
    results['ELECTORS'] = pd.Series(electors)
    if '%' in results:
        del results['%']
    return results.reset_index()

In [14]:
results = []
for filename in sorted(fieldlist):
    results.append(old_text_parse(filename))

results = pd.concat(results, ignore_index=True)['YEAR STATE PC NAME SEX PARTY SYMBOL AGE CATEGORY VOTES ELECTORS'.split(' ')]
print results

IOError: [Errno 2] No such file or directory: 'raw/2009.txt'

In [15]:
# Calculations
results['VOTES'] = results['VOTES'].astype(float)
results['#'] = results.groupby(['YEAR', 'STATE', 'PC'])['VOTES'].rank(method='min', ascending=False)
results.sort(['YEAR', 'STATE', 'PC', 'VOTES'], ascending=(True, True, True, False), inplace=True)
results.to_csv('parliament.csv', index=False, float_format='%.0f')

TypeError: list indices must be integers, not str